In [30]:
import pandas as pd
import os
os.getcwd()

'c:\\Users\\el_4\\OneDrive\\Документы\\Tilburg University\\Master DSS\\Thesis\\code\\Fact_Checking_model\\explore'

In [31]:
actors_df = pd.read_csv('../data/raw/MovieSummaries/character.metadata.tsv', delimiter='\t', header=None)
column_names = ['Wikipedia ID','Freebase movie ID','Movie release date', 'Character name', 'Actor dob', 'Gender', 'Height','Ethnicity','Actor','Age', 'Freebase character/actor map ID','Freebase character ID', 'Freebase actor ID']
actors_df.columns = column_names
actors_df.drop(['Movie release date', 'Character name', 'Actor dob', 'Gender', 'Height','Ethnicity','Age', 'Freebase character/actor map ID','Freebase character ID', 'Freebase actor ID'], axis=1, inplace=True)

print(actors_df)

        Wikipedia ID Freebase movie ID               Actor
0             975900         /m/03vyhn      Wanda De Jesus
1             975900         /m/03vyhn  Natasha Henstridge
2             975900         /m/03vyhn            Ice Cube
3             975900         /m/03vyhn       Jason Statham
4             975900         /m/03vyhn         Clea DuVall
...              ...               ...                 ...
450664        913762         /m/03pcrp  Dorothy Elias-Fahn
450665        913762         /m/03pcrp       Jonathan Fahn
450666      28308153        /m/0cp05t9      David Hemmings
450667      28308153        /m/0cp05t9    Roberta Paterson
450668      28308153        /m/0cp05t9         John Rogers

[450669 rows x 3 columns]


## Metadata


In [32]:
data = pd.read_csv('../data/raw/MovieSummaries/movie.metadata.tsv', delimiter='\t', header=None)
column_names = ['Wikipedia ID','Freebase movie ID','Movie name', 'Year', 'Movie box office revenue', 'Movie runtime', 'Movie languages', 'Movie countries', 'Genre']
data.columns = column_names
data.drop(['Freebase movie ID','Movie box office revenue', 'Movie runtime', 'Movie languages', 'Movie countries'], axis=1, inplace=True)

data['Genre'] = data['Genre'].apply(lambda x: eval(x).values())
data['Year'] = data['Year'].astype(str).str[:4]
data.loc[data['Year'] == '1010', 'Year'] = '2010'
#grouped_years = data.groupby('Year').size()
#grouped_years
df_combined = pd.merge(actors_df, data, on = 'Wikipedia ID')
df_combined.head(9)

def select_top_actors(group):
    # sort the group by actor's count and take the top 5
    top_actors = group.sort_values(by='Actor', ascending=False).head(5)
    return top_actors

# group the dataframe by the movie column and apply the select_top_actors function
df = df_combined.groupby('Movie name').apply(select_top_actors)

# reset the index to get rid of the groupby structure
df = df.reset_index(drop=True)

new_column_order = ['Wikipedia ID',	'Freebase movie ID','Movie name', 'Year', 'Genre', 'Actor']
df = df.reindex(columns=new_column_order)

print(df.head(7))